
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# 遅延初期化

:label: `sec_lazy_init`

これまでのところ、ネットワークの設定がずさんで済んだように見えるかもしれません。具体的には、次のような直感的ではないことを実行しましたが、機能するはずがないように見えるかもしれません。
- 入力次元を指定せずにネットワーク アーキテクチャを定義しました。
- 前のレイヤーの出力サイズを指定せずにレイヤーを追加しました。
- モデルに含めるパラメーターの数を決定するのに十分な情報を提供する前に、これらのパラメーターを「初期化」することもありました。

私たちのコードが実際に実行されることに驚かれるかもしれません。結局のところ、深層学習フレームワークがネットワークの入力次元が何になるかを知る方法はありません。ここでのトリックは、フレーム*ワークが初期化を延期し*、最初にモデルにデータを渡すまで待機して、各レイヤーのサイズをその場で推測することです。

その後、畳み込みニューラル ネットワークを使用する場合、入力の次元 (つまり、画像の解像度) が後続の各層の次元に影響を与えるため、この手法はさらに便利になります。したがって、コードの作成時に次元が何であるかを知らなくてもパラメーターを設定できる機能により、モデルを指定してその後変更するタスクが大幅に簡素化されます。次に、初期化の仕組みをさらに詳しく見ていきます。


In [1]:
import torch
from torch import nn
from d2l import torch as d2l


まず、MLP をインスタンス化しましょう。


In [2]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))


この時点では、入力次元が不明なままであるため、ネットワークは入力層の重みの次元を知ることができない可能性があります。



したがって、フレームワークはまだパラメータを初期化していません。以下のパラメータにアクセスして確認します。


In [3]:
net[0].weight

<UninitializedParameter>


次に、ネットワーク経由でデータを渡し、フレームワークが最終的にパラメータを初期化できるようにします。


In [4]:
X = torch.rand(2, 20)
net(X)

net[0].weight.shape

torch.Size([256, 20])


入力次元 20 がわかると、フレームワークは値 20 を代入することで最初の層の重み行列の形状を識別できます。最初の層の形状を認識すると、フレームワークは 2 番目の層に進み、以下同様に続きます。すべての形状が判明するまで計算グラフを作成します。この場合、最初の層のみ遅延初期化が必要ですが、フレームワークは順次初期化されることに注意してください。すべてのパラメータの形状が判明すると、フレームワークは最終的にパラメータを初期化できます。



次のメソッドは、すべてのパラメーター形状を推測するためのドライ ランのためにネットワークを介してダミー入力を渡し、その後パラメーターを初期化します。これは、後でデフォルトのランダムな初期化が望ましくない場合に使用されます。


In [5]:
@d2l.add_to_class(d2l.Module)  #@save
def apply_init(self, inputs, init=None):
    self.forward(*inputs)
    if init is not None:
        self.net.apply(init)


## まとめ

遅延初期化は便利で、フレームワークがパラメーターの形状を自動的に推測できるようになり、アーキテクチャの変更が容易になり、一般的なエラーの原因が 1 つ排除されます。モデルを介してデータを渡して、フレームワークが最終的にパラメーターを初期化できるようにします。

## 演習
1. 最初のレイヤーには入力寸法を指定し、後続のレイヤーには指定しなかった場合はどうなりますか?すぐに初期化されますか?
1. 一致しない寸法を指定するとどうなりますか?
1. さまざまな次元の入力がある場合、何をする必要があるでしょうか?ヒント: パラメーターの結合を見てください。



[ディスカッション](https://discuss.d2l.ai/t/8092)
